### Schedule

1. [~30 min] Intro presentation
2. [~60 min] Chapter 1 & 2 (Random & Mutate Loop)
3. [~15 min] Break
4. [~10 min] Discussion on warmup & compound difference
5. [~60 min] Chapter 3 (ML, ML Loop)
6. [~30 min] Chapter 4: Sending your best solution
7. [~15 min] Quick presentation by best performing solution

# Chapter 1: Random Screening In Known Compounds

### 📝 Meet our team

You were selected to be a lead medicinal chemist in a critical drug discovery project for our company. Your goal is to find a novel GSK3β inhibitor, which could be used to treat cancer. Your first idea is to buy a lot of diverse molecules found in the ZINC database and verify their biological activity in the lab. ZINC is one of the databases in which you can find small molecules that are available for purchase. Some other databases offer even more molecules that can be synthesized on demand. Let's select a random batch of molecules for testing to get started!

<img src="https://github.com/molecule-one/mlinpl-23-workshops/blob/main/assets/lab1.png?raw=true" width="400px" />

### 📘 Glossary

*Dopamine Receptor D$_2$ (DRD2)* - a G protein-coupled receptor that binds dopamine; a common target for antipsychotic drugs.

*Glycogen synthase kinase-3 beta (GSK3β)* - an enzyme that can be targeted to treat cancer.

*High-throughput screening (HTS)* - an experiment in which biological activity is tested automatically for many compounds in parallel.

*Inhibitor* - a molecule that blocks (inhibits) its biological target (usually a protein).

*Library* - a collection of molecules

*Medicinal chemistry* - a branch of chemistry that investigates the interactions between small molecules (or other compounds with potential therapeutic effects) and their biological targets, e.g. to learn how drugs work in the organism; this knowledge is often used to propose new molecules as drug candidates.

*Virtual screening (VS)* - an application of computational tools for finding active compounds in big virtual libraries of compounds.

*ZINC* - a database of readily purchasable compounds that can be used for virtual screening.

### ⚒️ Setup & Imports

Set the environment:
```
conda env create --file env.yml
```
And use de-novo-workshop for this notebook

In [ ]:
!conda env create --file env.yml

In [ ]:
!pip install PyTDC rdkit dataclasses_json mols2grid selfies

In [ ]:
from rdkit import Chem
import rdkit

In [ ]:
import os
your_path = ...
os.chdir(f"{your_path}/Automating_Science/molecules")
os.makedirs(your_path, exist_ok=True)

### ⚒️ Global Variables

In [ ]:
SERVER_URL = "https://orca-careful-remarkably.ngrok-free.app"
YOUR_TOKEN = ???  ### TODO: Paste your token here

### ⚒️ Exercise 1 Warmup: Requesting your first HTS experiment and achieve 28% score on GSK3B

Your task is to find early leads for GSK: achieving at least 28%. It will likely require tuning the budget of your experiments.

In [ ]:
# 1. Sample random compounds from the ZINC dataset

from molecules.src.compound_spaces import SmallZINC
from molecules.src.organic import show_molecules

budget = 500
library = SmallZINC()
candidates = library.sample(budget)
candidates_smiles = [rdkit.Chem.MolToSmiles(mol) for mol in candidates]
show_molecules(candidates)

In [ ]:
# 2. Request running HTS on the selected compounds
from src.al_loop import Loop, LeadCompound
from src.server_wrapper import FlaskAppClient

loop = Loop(base_dir=".", user_token=YOUR_TOKEN, target="GSK3β_server")
client = FlaskAppClient(base_url=SERVER_URL)

tested_molecules = loop.test_in_lab_and_save([LeadCompound(smiles=s) for s in set(candidates_smiles)], client=client)
show_molecules(tested_molecules)

In [ ]:
# 3. Show the summary of the screening results

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

data = pd.DataFrame({'Activity': [mol.activity for mol in tested_molecules]})
sns.histplot(data, binrange=(-1, 1), binwidth=0.01, x='Activity', log=True) # -1 corresponds to nonsynthesizable compounds
plt.show()

In [ ]:
# 4. Go to the leaderboard at https://orca-careful-remarkably.ngrok-free.app/leaderboard can you see your result?

In [ ]:
# 5. Code up random loop and run it!

from pathlib import Path
import shutil
from typing import List
from rich.console import Console
import numpy as np
console = Console()

class RandomLoop(Loop):
    """Samples random compounds from the ZINC database."""
    def __init__(self, base_dir: Path, user_token=None, target="GSK3β_server"):
        self.space = SmallZINC()
        super().__init__(base_dir, user_token, target)
    def propose_candidates(self, n_candidates: int) -> List[LeadCompound]:
        smi = []

        ### TODO: Create a list of randomly sampled SMILES strings.
        ### Hint: You can use code above.

        ### START SOLUTION
        ...

        ### END SOLUTION

        return [
            LeadCompound(s, None, None) for s in smi
        ]

In [ ]:
# Let's define a running function
def run(loop, budget=1000, purge=False, steps=10):
    target = loop.target
    user_token = loop.user_token

    if purge:
        shutil.rmtree(loop.base_dir, ignore_errors=True)

    os.makedirs(loop.base_dir, exist_ok=True)

    if target == "GSK3β":
        client = None
    else:
        client = FlaskAppClient(SERVER_URL)

    if loop.n_iterations > 0:
        raise ValueError(f"Already run. Please remove the folder {loop.base_dir} to run again.")

    metrics = []
    all_result: List[LeadCompound] = []
    budget_per_step = budget // steps
    assert budget % steps == 0 # for simplicity
    for step in range(steps):
        console.print(f"[red]Step {step}[/red]")
        candidates = loop.propose_candidates(budget_per_step)
        loop.test_in_lab_and_save(candidates, client=client)
        result: List[LeadCompound] = loop.load(iteration_id=step)
        all_result += result
        loop.generate_visualization(iteration_id=step)
        all_result_sorted = sorted(all_result, key=lambda x: x.activity, reverse=True)
        metrics.append({"top10": np.mean([x.activity for x in all_result_sorted[:10]]),
                        "top10_synth": np.mean([x.synth_score for x in all_result_sorted[:10]])})

    return metrics

In [ ]:
loop = RandomLoop(base_dir="random", user_token=YOUR_TOKEN, target="GSK3β_server")
metrics = run(loop, purge=True, budget=1000, steps=10)

In [ ]:
# 6. plot metrics using matplotlib
plt.plot([i*100 for i in range(len(metrics))],
          [m['top10'] for m in metrics], linewidth=4)
plt.grid()
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('N_compounds')
plt.ylabel('top10')
plt.show()

In [ ]:
# 7. This assert should work now!
assert metrics[-1]['top10'] > 0.25 # should match baseline-random on GSK

# Chapter 2: Go Beyond Known Compounds with Mutation

### 📝 Optimize the discovery process

Our initial screening campaign produced some positive results, but unfortunately, your manager is not satisfied with the cost-to-success ratio. A lot of money was spent on buying compounds and running biochemical assays, but only a few new compounds were found. It is crucial to optimize this process before the company runs out of funding. One possible solution is to use the already tested compounds to build a QSAR model that can aid in your search. Additionally, you can consider modifying the existing compounds to find similar compounds with better activity.

### 📘 Glossary

*Biochemical assay* - a test or an analytical procedure that measures protein binding or activity.

*ChEMBL* - a public database of molecule activity sourced from publications and patents.

*Molecular fingerprint* - a binary vector that encodes fragments that are included in the molecule; the fragments can be predefined or extracted automatically, e.g. by enumerating substructures around each atom of the molecule.

*Molecular graph* - a graph representation of a molecule, in which atoms are nodes and chemical bonds are edges, attributed with atom features that encode atom types (carbon, oxygen, nitrogen, etc.) and other atomic properties.

*QSAR* - quantitative structure-activity relationship; the name used to describe machine-learning models that predict activity based on the chemical structures at the input.

*Random forest* - a machine-learning model that produces its predictions by combining predictions of multiple decision trees constructed based on the input data; single decision trees perform a sequence of decisions on the input features to maximally separate different data classes (in our case, molecules with different activity labels).

*SELFIES* - a string representation of a molecule that was designed to work better with machine learning algorithms, e.g. by simplifying the grammar of possible molecules compared to SMILES strings and reducing the number of invalid structures.

*SMILES* - a string representation of a molecule that is commonly used to store chemical formulas in databases; in this representation, all atoms of the molecule are encoded by traversing the molecular graph using the DFS order.

### ⚒️ Exercise 2: Achieve >40% on GSK by exploring mutated compounds

<image of zombie>

Now we need to introduce a method that can optimize our molecules by introducing small structure modifications. We will use the SELFIES representation of molecules (linearized textual representation) that can be easily modified by changing some letters in the representation code. For example, we can change atoms by replacing their symbols in the SELFIES string. We can also add more atoms by adding symbols in the middle of the sequence.


<img src="https://github.com/molecule-one/mlinpl-23-workshops/blob/main/assets/lab4.png?raw=true" width="400px" />

In [ ]:
from selfies import encoder, decoder
import numpy as np
from src.mutate import mutate_selfie
from src.al_loop import LeadCompound
from src.organic import evaluate_synthesizability

In [ ]:
# found max activity
all_random_molecules = sum([loop.load(iteration_id=i) for i in range(loop.n_iterations)], [])
max([m.activity for m in all_random_molecules])

In [ ]:
mutate_top_k = 10
n_candidates = 100

### TODO: Get top k molecules with the best activity (as SELFIES)
### Hint: To encode SELFIES use `encoder(smiles)`
selfies = [...]

new_compounds = []
### TODO: Generate `n_candidates` new compounds by mutating top k molecules.
### Hint: To get SMILES of a mutated SELFIES use `decoder(mutate_selfie(selfies, max_molecules_len=100)[0])`
assert len(new_compounds) == n_candidates

loop = Loop(base_dir=".", user_token=YOUR_TOKEN, target="GSK3β_server")
client = FlaskAppClient(base_url=SERVER_URL)

tested_molecules = loop.test_in_lab_and_save([LeadCompound(smiles=s) for s in set(new_compounds)], client=client)
show_molecules(tested_molecules)

In [ ]:
# 3. Show the summary of the screening results

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

data = pd.DataFrame({'Activity': [mol.activity for mol in all_random_molecules]})
sns.histplot(data, binrange=(-1, 1), binwidth=0.01, x='Activity', log=True, color="blue", label="Random") # -1 corresponds to nonsynthesizable compounds

data2 = pd.DataFrame({'Activity': [mol.activity for mol in tested_molecules ]})
sns.histplot(data2, binrange=(-1, 1), binwidth=0.01, x='Activity', log=True, color="red", label="Mutate") # -1 corresponds to nonsynthesizable compounds

plt.legend()
plt.show()

In [ ]:
# 4. Implement mutation loop

class MutateLoop(Loop):
    """Implementation of AL algorithm that mutates top compounds from the previous iterations.

       Mutate loop should first search random and then mutate top compounds
    """
    def __init__(self, base_dir: Path, n_warmup_iterations: int = 1, mutate_top_k: int = 10, user_token=None, target="DRD2"):
        self.space = SmallZINC()
        self.n_warmup_iterations = n_warmup_iterations
        self.mutate_top_k = mutate_top_k
        super().__init__(base_dir, user_token, target)

    def _propose_random(self, n_candidates: int) -> List[LeadCompound]:
        smi = [self.space.try_sample()[0] for _ in range(n_candidates)]
        return [
            LeadCompound(s, None, None) for s in smi
        ]

    def propose_candidates(self, n_candidates: int) -> List[LeadCompound]:
        previous_results: List[LeadCompound] = self.load()

        if n_candidates < self.mutate_top_k:
            raise ValueError(f"n_candidates must be at least mutate_top_k ({self.mutate_top_k}).")

        if n_candidates == 0:
            return []

        if self.n_iterations < self.n_warmup_iterations:
            return self._propose_random(n_candidates)

        ### TODO: Implement mutation of top k compounds.
        ### Hint: You should have this part implemented above.

        ### START SOLUTION

        ...

        #### END SOLUTION

        assert len(new_compounds) == n_candidates
        return [LeadCompound(smiles=c) for c in new_compounds]

In [ ]:
# 5. Run mutation loop
mloop = MutateLoop(base_dir="mutate", user_token=YOUR_TOKEN, target="GSK3β_server")
mutate_metrics = run(mloop, purge=True, budget=1000, steps=10)

In [ ]:
# 5. Plot metrics using matplotlib
plt.plot([i*100 for i in range(len(mutate_metrics))],
          [m['top10'] for m in mutate_metrics], linewidth=4, label="mutate")
plt.plot([i*100 for i in range(len(metrics))],
          [m['top10'] for m in metrics], linewidth=4, label="random")
plt.legend()
plt.grid()
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('N_compounds')
plt.ylabel("Top 10")
plt.show()

In [ ]:
# 7. This assert should work now!
assert mutate_metrics[-1]['top10'] > 0.3 # should match baseline-random on GSK

In [ ]:
# 8. Check your performance on leaderboard. Do you see score > 0.4?

### 📝 Report your new findings

Congratulations! You found several novel molecules that are very promising! Now, you need to convince the stakeholders that your approach is highly efficient. Let's compare our new hits with random molecules sampled from ZINC

In [ ]:
# 9. Compare random and mutate. What do you observe?

top100_random = list(sorted(sum([loop.load(iteration_id=i) for i in range(loop.n_iterations)], []), key=lambda k: -k.activity))[0:100]
show_molecules(top100_random)

In [ ]:
top100_mutate = list(sorted(sum([mloop.load(iteration_id=i) for i in range(mloop.n_iterations)], []), key=lambda k: -k.activity))[0:100]
show_molecules(top100_mutate)

In [ ]:
# Examine the worst 100 compounds. These have assigned -1 activity, which are not synthesizable according to the laboratory
bottom100_mutate = list(sorted(sum([mloop.load(iteration_id=i) for i in range(mloop.n_iterations)], []), key=lambda k: k.activity))[0:100]
show_molecules(bottom100_mutate)

In [ ]:
# You can compute synthesizabiltiy before sending to the lab!
from src.sas_score import compute_ertl_score
compute_ertl_score(bottom100_mutate[0].smiles)

### ⚒️ Exercise 3: Tune the warmup.

Is is better to use longer or smaller warmup?

In [ ]:
# 5. Run mutation loop
mloop_it1 = MutateLoop(base_dir="mutate", n_warmup_iterations=1, user_token=YOUR_TOKEN, target="GSK3β_server")
mutate_metrics_it1 = run(mloop_it1, purge=True, budget=1000, steps=10)

mloop_it3 = MutateLoop(base_dir="mutate", n_warmup_iterations=3, user_token=YOUR_TOKEN, target="GSK3β_server")
mutate_metrics_it3 = run(mloop_it3, purge=True, budget=1000, steps=10)

# 5. Plot metrics using matplotlib
plt.plot([i*100 for i in range(len(mutate_metrics_it3))],
          [m['top10'] for m in mutate_metrics_it3], linewidth=4, label="mutate (it=3)")
plt.plot([i*100 for i in range(len(mutate_metrics_it1))],
          [m['top10'] for m in mutate_metrics_it1], linewidth=4, label="mutate (it=1)")
plt.legend()
plt.grid()
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('N_compounds')
plt.ylabel('top10')
plt.show()

# Chapter 3: Machine Learning To Select Compounds More Efficiently



### 📝 Automate the process

Your reports drew the attention of stakeholders, and now the company asks you if you can make more accurate predictions, as many compounds end up inactive after testing in the laboratory. Adding machine Learning is a natural idea.

Good luck!

### ⚒️ Exercise 4: Achieve meaningful correlation in molecular property prediction

We will build a simple QSAR model based on the compounds that we tested in our previous experiment. In your other projects, you could also use publicly available data. Databases such as ChEMBL offer a lot of activity data gathered from various online sources.

In the following experiment, we will use Morgan fingerprints to encode molecules. We will use them as an input to a random forest model that predicts the experimental activity. After training this model, we can use it to predict activity without paying for performing biochemical testing in a wet lab.


<img src="https://github.com/molecule-one/mlinpl-23-workshops/blob/main/assets/lab2.png?raw=true" width="400px" />

In [ ]:
# Let's load all mutated compounds
mutate_cmpds = sum([mloop.load(iteration_id=i) for i in range(mloop.n_iterations)], [])

# And build dataset
data = pd.DataFrame({
    'activity': [mol.activity for mol in mutate_cmpds],
    'smiles': [mol.smiles for mol in mutate_cmpds],
    'synthesizability': [mol.synth_score for mol in mutate_cmpds],
})

In [ ]:
# We need to featurize compounds
from rdkit.Chem import AllChem
import numpy as np

def calculate_fingerprint(smiles: str) -> np.ndarray:
    ### TODO: compute a fingerprint
    return ...

data['fingerprint'] = data.smiles.apply(calculate_fingerprint)
data = data[data['activity']>=0]
data.head(10)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming data is already loaded
X_data, y_data = np.stack(data['fingerprint']), data.activity

# Split the data into training and testing sets with a given proportion, for example, 80% for training and 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)

In [ ]:
df_preds = pd.DataFrame({'preds': model.predict(X_test), 'labels': y_test})
corr = df_preds.corr('spearman').iloc[1, 0]
print("ρ = ", corr)
sns.scatterplot(data=df_preds, x='preds', y='labels')

In [ ]:
assert corr > 0.8

### Optional: Graph neural networks

You can also use graph neural networks.

<img src="https://github.com/molecule-one/mlinpl-23-workshops/blob/main/assets/lab3.png?raw=true" width="400px" />

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise ValueError("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def featurize_graph(smiles: str):
    mol = Chem.MolFromSmiles(smiles)

    A = np.zeros((mol.GetNumAtoms(), mol.GetNumAtoms()))
    for bond in mol.GetBonds():
      ### TODO: Add bonds to the adjacency matrix
      ...

    nodes = []
    for atom in mol.GetAtoms():
        ### TODO: Improve the atom encoding suggested below
        results = one_of_k_encoding_unk(
            atom.GetSymbol(),
            ['C', 'O', 'N', 'Cl', 'F', 'S', '']
        )
        nodes.append(results)
    X = np.array(nodes).astype(float)

    return X, A

In [ ]:
X, A = featurize_graph('c1ccccc1')
assert X.shape[0] == 6
assert A.shape == (6, 6)

In [ ]:
def collate_graphs(batch):
  X_all = []
  A_all = []
  for data in batch:
    X, A = featurize_graph(data.smiles)
    X_all.append(X)
    A_all.append(A)
  max_size = max([A.shape[0] for A in A_all])
  A_padded = np.zeros((max_size * len(X_all), max_size))
  for i, A in enumerate(A_all):
    A_padded[i * max_size:i * max_size + A.shape[0], 0:A.shape[0]] = A
  X_padded = np.zeros((max_size * len(X_all), X_all[0].shape[1]))
  for i, X in enumerate(X_all):
    X_padded[i * max_size:i * max_size + X.shape[0], :] = X
  return (
      torch.Tensor(X_padded),
      torch.Tensor(A_padded),
      torch.Tensor([data.activity for data in batch]),
  )

In [ ]:
data = [cmpd for cmpd in mutate_cmpds if cmpd.activity >= 0]

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)
train_loader = DataLoader(
    data_train,
    batch_size=4,
    collate_fn=collate_graphs,
    shuffle=True,
)
test_loader = DataLoader(
    data_test,
    batch_size=4,
    collate_fn=collate_graphs,
    shuffle=False,
)

In [ ]:
import math

def mean_pool(X, batch_size):
  pool = X.view(batch_size, -1, X.shape[1])
  return pool.mean(dim=1)

class GCNLayer(torch.nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.W = torch.nn.parameter.Parameter(torch.empty((in_features, out_features)))
    torch.nn.init.kaiming_uniform_(self.W, a=math.sqrt(5))

  def forward(self, X, A):
    size = A.shape[1]
    X_new = []
    A = A.view(-1, size, size)
    X = X.view(-1, size, X.shape[1])
    for adj, x in zip(A, X):
      ### TODO: Implement graph convolution
      X_new.append(...)
    return torch.concatenate(X_new)

class GraphNeuralNetwork(torch.nn.Module):
  def __init__(self, hidden_size, input_size=7):
    super().__init__()
    ### TODO: Write your GNN architecture
    ...

  def forward(self, batch):
    ### TODO: Write your GNN architecture
    X, A, y = batch
    batch_size = len(y)
    ...

In [ ]:
from tqdm.notebook import tqdm, trange

def train(train_loader):
    ### TODO: define and choose your hyperparameters
    # hyperparameters definition
    hidden_size = ...
    epochs = ...
    learning_rate = ...

    # model preparation
    model = GraphNeuralNetwork(hidden_size)
    model.train()

    # training loop
    optimizer = ... ### TODO: define an optimizer
    loss_fn = ...  ### TODO: define a loss function
    for epoch in trange(1, epochs + 1, leave=False):
        for data in tqdm(train_loader, leave=False):
            y = data[2]
            model.zero_grad()
            preds = model(data)
            loss = loss_fn(preds, y.reshape(-1, 1))
            loss.backward()
            optimizer.step()
    return model


def predict(model, test_loader):
    # evaluation loop
    preds_batches = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            preds = model(data)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds


# training
model = train(train_loader)

In [ ]:
df_preds = pd.DataFrame({
    'preds': predict(model, test_loader).flatten(),
    'labels': [cmpd.activity for cmpd in data_test],
})
corr = df_preds.corr('spearman').iloc[1, 0]
print("ρ = ", corr)
sns.scatterplot(data=df_preds, x='preds', y='labels')

In [ ]:
assert corr > 0.75

### ⚒️ Exercise 5: Machine Learning Based Loop

In [ ]:
# 1. Initialize a base loop that mutates compounds based on the input candidates
base_loop = MutateLoop(
    base_dir='mlloop',
    n_warmup_iterations=3,
    user_token=YOUR_TOKEN,
    target='GSK3β_server',
)

In [ ]:
# 2. Implement active learning loop that mutates compounds and filters the candidates using ML predictive models

from pathlib import Path
from typing import List

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


class MLLoop(Loop):
    """
    Your final implementation of the experimental loop.

    The algorithm you implement in the `propose_candidates` method will be repeated
    several times to iteratively improve your candidates.

    The molecules will be sent to the official lab endpoint with a LIMITED NUMBER OF REQUESTS,
    so use this code wisely and care for the synthesizability of your compounds!
    """
    def __init__(self, base_dir: Path, base_loop: Loop, n_warmup_iterations: int=1, user_token=None, target="DRD2"):
        self.base_loop = base_loop
        self.n_warmup_iterations = n_warmup_iterations
        super().__init__(base_dir, user_token, target)

    def _featurize(self, smi: List[str]) -> np.ndarray:
        ### TODO: write up featurizer
        # START SOLUTION
        ...
        # END SOLUTION

    def _split(self, X, y, smi) -> ...:
        X_temp, X_test, y_temp, y_test, smi_temp, smi_test = \
            train_test_split(X, y, smi, test_size=0.3, random_state=42)
        X_train, X_valid, y_train, y_valid, smi_train, smi_valid = \
            train_test_split(X_temp, y_temp, smi_temp, test_size=0.3, random_state=42)
        return X_train, X_valid, X_test, y_train, y_valid, y_test, smi_train, smi_valid, smi_test

    def _train_model(self, previous_results: List[LeadCompound]):
        """Trains models and assigns to ._model variable."""
        previous_results = [c for c in previous_results if c.activity != -1]
        if len(previous_results) == 0:
            raise ValueError("No previous results to train on (excluded activity = -1). Perhaps your "
                             "base loop proposes nonsynthetizable compounds?")
        console.log(f"Retraining model on {len(previous_results)} compounds.")
        smi = [c.smiles for c in previous_results]
        X = self._featurize(smi)
        y = np.array([c.activity for c in previous_results])
        y = y > np.median(y) # convert to binary. loses information naturally.

        console.log(f"Shapes: X {X.shape}, y {y.shape}")

        # split using sklearn
        X_train, X_valid, X_test, y_train, y_valid, y_test, smi_train, smi_valid, smi_test = self._split(X, y, smi)

        console.log(f"Training set size: {len(X_train)}")
        console.log(f"Validation set size: {len(X_valid)}")
        console.log(f"Test set size: {len(X_test)}")
        console.log(f"Training set activity mean: {np.mean(y_train)}")
        console.log(f"Proceeding to training Random Forest")
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)

        self._model = rf

    def _select_top_N(self, candidates: List[LeadCompound], n_select: int) -> List[LeadCompound]:
        """Ranks candidates by their predicted activity."""
        X_test = self._featurize([c.smiles for c in candidates])
        y_pred = self._model.predict_proba(X_test)
        if y_pred.ndim == 2:
            y_pred = y_pred[:, -1]
        return [c for _, c in sorted(zip(y_pred, candidates), reverse=True, key=lambda a: a[0])][:n_select]

    def propose_candidates(self, n_candidates: int) -> List[LeadCompound]:
        if self.n_iterations < self.n_warmup_iterations:
            return self.base_loop.propose_candidates(n_candidates)

        previous_results: List[LeadCompound] = self.load()
        ### TODO: fill the missing code

In [ ]:
mlloop = MLLoop(
    base_dir="mlloop",
    base_loop=base_loop,
    user_token=YOUR_TOKEN,
    target="GSK3β_server",
)
ml_metrics = run(mlloop, purge=True)

# 5. Plot metrics using matplotlib
plt.plot([i*100 for i in range(len(ml_metrics))],
          [m['top10'] for m in ml_metrics], linewidth=4, label="ML")
plt.plot([i*100 for i in range(len(mutate_metrics_it3))],
          [m['top10'] for m in mutate_metrics_it3], linewidth=4, label="mutate (it=3)")
plt.plot([i*100 for i in range(len(mutate_metrics_it1))],
          [m['top10'] for m in mutate_metrics_it1], linewidth=4, label="mutate (i=1)")
plt.legend()
plt.grid()
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('N_compounds')
plt.ylabel('top10')
plt.show()

# Chapter 4: Finding drugs for all targets

The final part of the workshop is to find miracle drugs for DRD2 and JNK3 targets.

Some tips:

1. Pay attention to warmup.
2. Pay attention to synthesizability. You can compute synthesizabiltiy before sending to the lab!

```
from src.sas_score import compute_ertl_score;
compute_ertl_score(bottom100_mutate[0].smiles)```

3. Pay attention to whether you can trust your model (e.g. you might want to select very confident prediction if your model is calibrated).

In [ ]:
# your code